# Search Terms Analysis

*Neal Xie*

Marketing is inherently analytic. Field-testing marketing  outreach and marketing performance is key to optimizing budget allocaion and market reach. Search marketing is no different, and since keywords dictate our entire search campaign, **Keyword Analysis** should be your primary focus.

This analysis uses Diageo - Smirnoff brand data camparing with May 2017 and Nov 2017 to show case that how to transfer keyword data to topic level and make optimizations.

In [27]:
# Import everything I am going to need up front
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.externals import joblib

In [3]:
# Get csv file for search term report
search_term = pd.read_csv('C:/_Project/Diageo_Search/Search_term_report_last30.csv', sep = "\t")

In [4]:
# Take a look at first 5 rows
search_term.head()

,Match type,Search term,Added/Excluded,Campaign,Ad group,Keyword,Clicks,Impressions,CTR,Avg. CPC,Cost,Avg. position
0,exact (close variant),pineapple cocktails with vodka,None,C: Pineapple,Pineapple,[pineapple vodka cocktail],0,1,0.00%,0.00,0,1.0
1,exact (close variant),white rushin,None,R: White Russian,White Russian,[white russian],2,9,22.22%,0.42,0.84,1.1
2,exact (close variant),thanks giving cocotial,None,L: Thanksgiving,Thanksgiving Cocktail - Exact,[thanksgiving cocktail],1,1,100.00%,1.57,1.57,2.0
3,exact (close variant),alcoholic thanksgiving drink,None,L: Thanksgiving,Thanksgiving-Drink,[thanksgiving alcoholic drink],2,12,16.67%,3.13,6.26,1.3
4,exact,spiked punch for thanksgiving,Added,L: Thanksgiving,Thanksgiving-Punch,[spiked punch for thanksgiving],1,1,100.00%,0.38,0.38,1.0


In [7]:
# Take a look at data types
search_term.dtypes

Match type         object
Search term        object
Added/Excluded     object
Campaign           object
Ad group           object
Keyword            object
Clicks              int64
Impressions         int64
CTR                object
Avg. CPC          float64
Cost              float64
Avg. position     float64
dtype: object

In [6]:
# Change Cost from object to float and clean it
search_term['Cost'] = search_term['Cost'].str.replace(',','').astype(np.float64)

In [8]:
# Creative a list of search term
search_term_list = list(search_term['Search term'])

In [9]:
# Take a look at first 10 rows of search term list
search_term_list[:10]

['pineapple cocktails with vodka',
 'white rushin',
 'thanks giving cocotial',
 'alcoholic thanksgiving drink',
 'spiked punch for thanksgiving',
 'spiked punch for thanksgiving',
 'what drinks can you make with blueberry vodka',
 'smirnoff peppermint bark martini',
 'white russisns',
 'how to make a dirt martini']

In [10]:
# Load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
# Print 10 stopwords
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [12]:
# Here I define a tokenizer which returns the set of stems in the text that is is passed
def tokenize_only(text):
    # First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # Filter out any tokens not containing letters (e.g., numeric tokens, raw punctiation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [13]:
# Use extend so it's a big flat list of vacab
totalvocab_tokenized = []
for i in search_term_list:
    allwords_tokenized = tokenize_only(i) # For each item in 'search_term_list', tokenized
    totalvocab_tokenized.extend(allwords_tokenized) # Extend the 'totalvocab_tokenized' list

In [17]:
# Take a look at first 10 rows of totalvocab_tokenized
totalvocab_tokenized[:10]

['pineapple',
 'cocktails',
 'with',
 'vodka',
 'white',
 'rushin',
 'thanks',
 'giving',
 'cocotial',
 'alcoholic']

In [18]:
# Create count and tf-idf vectorizer matrix
# Define count vectorizer parameters
count_vectorizer = CountVectorizer(max_df = 0.7,
                                   min_df = 3,
                                   max_features = None,
                                   stop_words = 'english',
                                   strip_accents = 'unicode',
                                   analyzer = 'word',
                                   tokenizer = tokenize_only,
                                   ngram_range = (1,3)
                                   )

# Define tfidf vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df = 0.7,
                                   min_df = 3,
                                   max_features = None,
                                   stop_words = 'english',
                                   strip_accents = 'unicode',
                                   analyzer = 'word',
                                   use_idf = True,
                                   tokenizer = tokenize_only,
                                   ngram_range = (1,3)
                                  )

%time tfidf_matrix = tfidf_vectorizer.fit_transform(search_term_list) # Fit the tfidf vectorizer to search term list
%time count_matrix = count_vectorizer.fit_transform(search_term_list) # Fit the count vectorizer to search term list

tfidf_matrix.shape
count_matrix.shape

Wall time: 1.43 s
Wall time: 1.34 s


(8571, 2051)

In [19]:
# Define a list of the feature used in the count matrix
count = count_vectorizer.get_feature_names()

In [20]:
# Create a count dataframe with feature names
count_df = pd.DataFrame(count_matrix.toarray(), columns=count)

In [21]:
# Take a look at first 5 rows of count_df
count_df.head(5)

,'s,'s good,'s good mix,'s smirnoff,'s white,'s white russian,100proof,4th,4th july,50ml,...,wins,winter,winter cocktails,wirh,wjite,world,woth,www,www smirnoff,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Sum tokens based on count and sort ascending
count_df.sum(0).sort_values(ascending=False)

smirnoff                     3066
vodka                        2664
drinks                        687
drink                         644
thanksgiving                  557
recipe                        546
peppermint                    527
recipes                       523
martini                       458
make                          447
white                         411
caramel                       401
mix                           344
russian                       287
apple                         284
cocktail                      283
watermelon                    277
caramel vodka                 263
cocktails                     251
raspberry                     244
vanilla                       238
white russian                 220
strawberry                    216
smirnoff peppermint           206
mule                          201
peppermint vodka              199
punch                         183
vanilla vodka                 181
good                          179
green         

In [23]:
# Define a list of features used in the tf-idf matrix
terms = tfidf_vectorizer.get_feature_names()

In [26]:
# K-means clustering
num_clusters = 12 # Number of clusters

km = KMeans(n_clusters = num_clusters) # Initial model

%time km.fit(tfidf_matrix) # Fit model

clusters = km.labels_.tolist() # Create cluster list

Wall time: 4.36 s


In [ ]:
# Save your model to pickle file
joblib.dmp(km, 'doc_cluster.pkl')

# Load your model from the pickle
km = joblib.load('doc_cluster.pkl')

In [28]:
# Print all clusters associated with thier search terms
from __future__ import print_function

print("Top terms per cluster:")
print()
# sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :12]: # replace 12 with n words per cluster
        print(' %s' % terms[ind], end=',')
        print()
    print() # add whitespace
    print() # add whitespace
    
    print("Cluster %d Search term:" % i, end='')
    for term in search_term.ix[i:12]['Search term']:
        print(' %s,' % term, end='')
        print()
    print() #add whitespace
    print() #add whitespace

print()
print()

Top terms per cluster:

Cluster 0 words: peppermint,
 smirnoff peppermint,
 peppermint vodka,
 smirnoff,
 peppermint twist,
 twist,
 peppermint smirnoff,
 vodka,
 smirnoff peppermint twist,
 smirnoff peppermint vodka,
 mix,
 mix peppermint,


Cluster 0 Search term: pineapple cocktails with vodka,
 white rushin,
 thanks giving cocotial,
 alcoholic thanksgiving drink,
 spiked punch for thanksgiving,
 spiked punch for thanksgiving,
 what drinks can you make with blueberry vodka,
 smirnoff peppermint bark martini,
 white russisns,
 how to make a dirt martini,
 smirnfoff limited edition,
 boueberry vodka,
 how much alcohol smirnoff watermelon ice,


Cluster 1 words: vodka,
 drinks,
 caramel,
 caramel vodka,
 vodka drinks,
 recipes,
 vanilla,
 vanilla vodka,
 vodka recipes,
 smirnoff vodka,
 smirnoff,
 mix,


Cluster 1 Search term: white rushin,
 thanks giving cocotial,
 alcoholic thanksgiving drink,
 spiked punch for thanksgiving,
 spiked punch for thanksgiving,
 what drinks can you make wi

In [29]:
# Add cluster column to search_term table
search_term['Cluster'] = clusters

In [30]:
# Print search terms count group by clusters
search_term['Cluster'].value_counts()

8     3461
1     1637
0      477
2      457
9      441
7      439
4      392
5      389
10     311
11     210
6      183
3      174
Name: Cluster, dtype: int64

In [33]:
# Print impressions group by clusters
grouped_impressions = search_term['Impressions'].groupby(search_term['Cluster']) # Group by cluster for aggregation purposes
grouped_impressions.sum() # Sum impressions per cluster

Cluster
0      14349
1     141008
2      68639
3      17552
4      16076
5      63437
6     150499
7      41992
8     117591
9     104724
10    118199
11     20120
Name: Impressions, dtype: int64

In [34]:
grouped_impressions.mean() # Average impressions per cluster

Cluster
0      30.081761
1      86.138057
2     150.194748
3     100.873563
4      41.010204
5     163.077121
6     822.398907
7      95.653759
8      33.976018
9     237.469388
10    380.061093
11     95.809524
Name: Impressions, dtype: float64

In [35]:
# Print clicks group by clusters
grouped_clicks = search_term['Clicks'].groupby(search_term['Cluster']) # Group by cluster for aggregation purposes
grouped_clicks.sum() # Sum clicks per cluster

Cluster
0      4299
1     16088
2      5110
3      2108
4      3667
5     11281
6     31136
7      7880
8     18654
9     13485
10    22285
11     3129
Name: Clicks, dtype: int64

In [36]:
grouped_clicks.mean() # Average clicks per cluster

Cluster
0       9.012579
1       9.827734
2      11.181619
3      12.114943
4       9.354592
5      29.000000
6     170.142077
7      17.949886
8       5.389772
9      30.578231
10     71.655949
11     14.900000
Name: Clicks, dtype: float64

In [37]:
# Print cost group by clusters
grouped_cost = search_term['Cost'].groupby(search_term['Cluster']) # Group by cluster for aggregation purposes
grouped_cost.sum() # sum cost per cluster

Cluster
0      1503.61
1     23756.69
2      3801.46
3      2041.59
4      5731.81
5     16623.71
6     13931.26
7      6445.89
8     18510.45
9     19174.52
10     9309.78
11     4304.34
Name: Cost, dtype: float64

In [38]:
grouped_cost.mean() # Average cost per cluster

Cluster
0      3.152222
1     14.512334
2      8.318293
3     11.733276
4     14.621964
5     42.734473
6     76.127104
7     14.683121
8      5.348295
9     43.479637
10    29.934984
11    20.496857
Name: Cost, dtype: float64

In [40]:
# Create final table to show wordcloud
# Join count table with search_term table
df_final = search_term.join(count_df, lsuffix = '_search_term', rsuffix = '_count_df')

# Use count multiply impressions to get total impression per term
df_wordcloud = count_df.multiply(df_final['Impressions'],axis=0).join(df_final['Cluster'].to_frame())

In [41]:
# Set cluster as index
df_wordcloud = df_wordcloud.set_index('Cluster')
cluster0 = df_wordcloud[df_wordcloud.index ==0].sum(0).sort_values(ascending=False).dropna()
cluster1 = df_wordcloud[df_wordcloud.index ==1].sum(0).sort_values(ascending=False).dropna()
cluster2 = df_wordcloud[df_wordcloud.index ==2].sum(0).sort_values(ascending=False).dropna()
cluster3 = df_wordcloud[df_wordcloud.index ==3].sum(0).sort_values(ascending=False).dropna()
cluster4 = df_wordcloud[df_wordcloud.index ==4].sum(0).sort_values(ascending=False).dropna()
cluster5 = df_wordcloud[df_wordcloud.index ==5].sum(0).sort_values(ascending=False).dropna()
cluster6 = df_wordcloud[df_wordcloud.index ==6].sum(0).sort_values(ascending=False).dropna()
cluster7 = df_wordcloud[df_wordcloud.index ==7].sum(0).sort_values(ascending=False).dropna()
cluster8 = df_wordcloud[df_wordcloud.index ==8].sum(0).sort_values(ascending=False).dropna()
cluster9 = df_wordcloud[df_wordcloud.index ==9].sum(0).sort_values(ascending=False).dropna()
cluster10 = df_wordcloud[df_wordcloud.index ==10].sum(0).sort_values(ascending=False).dropna()
cluster11 = df_wordcloud[df_wordcloud.index ==11].sum(0).sort_values(ascending=False).dropna()

In [ ]:
# Wordcloud
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image
from collections import Counter

mask = np.array(Image.open("smirnoff_bg.jpg"))
image_colors = ImageColorGenerator(mask)
wc = WordCloud(background_color = "white", mask = mask, color_func = image_colors)
wc.generate_from_frequencies(dict(np.log(cluster0+1.0001))) # Change cluster number to get wordcloud
wc.to_file("cluster0.png")

In [42]:
# Run Regression model 
import statsmodels.formula.api as sm

search_term['Cluster'] = search_term['Cluster'].astype('str')
search_term['Cost_log'] = np.log(search_term['Cost']+1)

result = sm.ols(formula = "Cost_log ~ Cluster", data = search_term).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               Cost_log   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     26.52
Date:                Tue, 02 Jan 2018   Prob (F-statistic):           4.82e-55
Time:                        15:49:35   Log-Likelihood:                -12255.
No. Observations:                8571   AIC:                         2.453e+04
Df Residuals:                    8559   BIC:                         2.462e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept         0.5471      0.046     11.811